## Git stuff


In [1]:
! git clone 'https://github.com/aassegai/Kontur23'
import os
os.chdir('Kontur23')

Cloning into 'Kontur23'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 204 (delta 73), reused 153 (delta 35), pack-reused 0
Receiving objects: 100% (204/204), 1.28 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (73/73), done.


## imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=5dfee7fc85969348f58e696f42deec59f278a4319a982392638b70bc555e47fa
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [4]:
! pip install spacy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
! pip install spacy[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.1 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import spacy
from spacy import displacy
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from src.model.predict import predict_entities
from src.data_preprocessing.prepare_bin import prepare_bin
from src.data_preprocessing.data_preprocessor import DataPreprocessor


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
'''
Sometimes when GPU runtime is used an error appears 
during spacy installation
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
This piece of code fixes this problem
'''
import locale
if str(locale.getpreferredencoding()) != 'UTF-8':
    def getpreferredencoding(do_setlocale = True):
      return "UTF-8"
    locale.getpreferredencoding = getpreferredencoding

In [8]:
! python -m spacy download ru_core_news_sm

2023-04-22 04:40:13.376028: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


# In this notebook we will do all the steps from the main notebook but with different options for data preprocessing. 

## Data retrieval

In [9]:
train_df = pd.read_json('./data/raw/train.json')
train_df, val_df = train_test_split(train_df, test_size=0.2)
val_df.set_index('id', inplace=True)
for idx, row in val_df.iterrows():
    val_df.loc[idx].extracted_part['label'] = row.label

In [10]:
test_df = pd.read_json('./data/raw/test.json')

## Normalization_tests


In [11]:
normalizing_data_preprocessor = DataPreprocessor(lemmatize=True)

In [12]:
train_normalized_df = normalizing_data_preprocessor(train_df, set_index_to_id=True)

  0%|          | 0/1439 [00:00<?, ?it/s]

In [13]:
for idx, row in train_normalized_df.iterrows():
    train_normalized_df.loc[idx].extracted_part['label'] = row.label

In [14]:
train_normalized_df

,text,label,extracted_part
id,,,
131828382,1 13 проект договор договор № на выполнение по...,обеспечение исполнения контракта,"{'answer_start': [1211], 'answer_end': [1317],..."
550453993,приложение № 1 к извещение о осуществление зак...,обеспечение исполнения контракта,"{'answer_start': [1240], 'answer_end': [1358],..."
17388699,извещение № зк-2022-09-699 о проведение запрос...,обеспечение исполнения контракта,"{'answer_start': [1100], 'answer_end': [1196],..."
792964855,«утверждаю» главный врач гуз «саратовский обла...,обеспечение исполнения контракта,"{'answer_start': [1218], 'answer_end': [1301],..."
67082098,11 проект государственный контракт № _______ н...,обеспечение гарантийных обязательств,"{'answer_start': [0], 'answer_end': [0], 'text..."
...,...,...,...
228373439,идентификационный код закупки: 222633005016063...,обеспечение исполнения контракта,"{'answer_start': [1286], 'answer_end': [1396],..."
658006780,договор № ________ г. москва «___» __________ ...,обеспечение гарантийных обязательств,"{'answer_start': [1211], 'answer_end': [1351],..."
419218330,идентификационный код закупки: 222633005042763...,обеспечение исполнения контракта,"{'answer_start': [1285], 'answer_end': [1395],..."


In [ ]:
prepare_bin(train_normalized_df, option='normalized')
prepare_bin(val_df, mode='val')
prepare_bin(test_df, mode='test')

0it [00:00, ?it/s]

270 items had blank annotation and were skipped


0it [00:00, ?it/s]

72 items had blank annotation and were skipped


0it [00:00, ?it/s]

0 items had blank annotation and were skipped


In [ ]:
! python -m spacy init fill-config ./src/model/spacy_cfg.cfg config.cfg

2023-04-21 02:42:46.211890: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./normalization_test/ --paths.train ./normalized_train_dataset.spacy --paths.dev ./val_dataset.spacy --gpu-id 0 

2023-04-21 02:43:02.017823: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: normalization_test
ℹ Saving to output directory: normalization_test
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-21 02:43:12,560] [INFO] Set up nlp object from config
[2023-04-21 02:43:12,571] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-21 02:43:12,574] [INFO] Created vocabulary
[2023-04-21 02:43:12,575] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS

We can see that quality on validation set with words lemmatizing is **~25% lower** than without it. Let's look at some examples.

In [ ]:
nlp_ner_normalization = spacy.load('./normalization_test/model-best')

In [ ]:
test_df

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта
...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств


In [ ]:
test_df.label.loc[9]

'обеспечение исполнения контракта'

In [ ]:
doc = nlp_ner_normalization(test_df.text.loc[9])

colors = {"обеспечение исполнения контракта": "#FF2D00", "обеспечение гарантийных обязательств": "#36FF00"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Here the model performs ok but on the next example...

In [ ]:
test_df.label.loc[310]

'обеспечение гарантийных обязательств'

In [ ]:
doc = nlp_ner_normalization(test_df.text.loc[310])

colors = {"обеспечение исполнения контракта": "#FF2D00", "обеспечение гарантийных обязательств": "#36FF00"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


Of course annotations label does not match the requested but that's bad anyway if things like this will happen on the inference.

## Stopwords tests


In [ ]:
stopwords_data_preprocessor = DataPreprocessor(remove_stopwords=True)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_stopwords_df = stopwords_data_preprocessor(train_df, set_index_to_id=True)

  0%|          | 0/1439 [00:00<?, ?it/s]

In [ ]:
for idx, row in train_stopwords_df.iterrows():
    train_stopwords_df.loc[idx].extracted_part['label'] = row.label

In [ ]:
train_stopwords_df

,text,label,extracted_part
id,,,
351152517,ПРИЛОЖЕНИЕ К ИЗВЕЩЕНИЮ 1. жетной классификации...,обеспечение исполнения контракта,"{'answer_start': [995], 'answer_end': [1065], ..."
892721516,Договор поставки №___________________________ ...,обеспечение гарантийных обязательств,"{'answer_start': [0], 'answer_end': [0], 'text..."
215713905,9 Проект государственного контракта № ________...,обеспечение исполнения контракта,"{'answer_start': [1230], 'answer_end': [1339],..."
567219201,Извещение проведении электронного аукциона зак...,обеспечение гарантийных обязательств,"{'answer_start': [1198], 'answer_end': [1268],..."
452496091,Извещение проведении электронного аукциона зак...,обеспечение гарантийных обязательств,"{'answer_start': [1235], 'answer_end': [1304],..."
...,...,...,...
334674371,Приложение №2 извещению проведении электронног...,обеспечение исполнения контракта,"{'answer_start': [1190], 'answer_end': [1291],..."
418909970,Извещение проведении электронного аукциона зак...,обеспечение гарантийных обязательств,"{'answer_start': [1206], 'answer_end': [1276],..."
859637373,Извещение проведении открытого конкурса электр...,обеспечение исполнения контракта,"{'answer_start': [1226], 'answer_end': [1272],..."


In [ ]:
prepare_bin(train_stopwords_df, option='stopwords')

0it [00:00, ?it/s]

236 items had blank annotation and were skipped


In [ ]:
! python -m spacy init fill-config ./src/model/spacy_cfg.cfg config.cfg

2023-04-21 04:10:53.038398: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./stopwords_test/ --paths.train ./stopwords_train_dataset.spacy --paths.dev ./val_dataset.spacy --gpu-id 0 

2023-04-21 04:11:04.974994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: stopwords_test
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-21 04:11:11,260] [INFO] Set up nlp object from config
[2023-04-21 04:11:11,277] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-21 04:11:11,282] [INFO] Created vocabulary
[2023-04-21 04:11:11,283] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from th

Stopwords removal makes quality a bit worse.

In [ ]:
nlp_ner_stopwords = spacy.load('./stopwords_test/model-best')

In [ ]:
test_df.label.loc[310]

'обеспечение исполнения контракта'

In [ ]:
doc = nlp_ner_stopwords(test_df.text.loc[310])

colors = {"обеспечение исполнения контракта": "#FF2D00", "обеспечение гарантийных обязательств": "#36FF00"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In comparison, this model does not see any strange entities in this document.

## Punctuation tests


In [ ]:
punctuation_data_preprocessor = DataPreprocessor(remove_punctuation=True)

In [ ]:
train_punctuation_df = punctuation_data_preprocessor(train_df, set_index_to_id=True)

  0%|          | 0/1439 [00:00<?, ?it/s]

In [ ]:
for idx, row in train_punctuation_df.iterrows():
    train_punctuation_df.loc[idx].extracted_part['label'] = row.label

In [ ]:
train_punctuation_df

,text,label,extracted_part
id,,,
351152517,ПРИЛОЖЕНИЕ К ИЗВЕЩЕНИЮ 1 жетной классификации ...,обеспечение исполнения контракта,"{'answer_start': [991], 'answer_end': [1058], ..."
892721516,Договор поставки ___________________________ г...,обеспечение гарантийных обязательств,"{'answer_start': [0], 'answer_end': [0], 'text..."
215713905,9 Проект государственного контракта __________...,обеспечение исполнения контракта,"{'answer_start': [1259], 'answer_end': [1370],..."
567219201,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1252], 'answer_end': [1322],..."
452496091,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1254], 'answer_end': [1323],..."
...,...,...,...
334674371,Приложение 2 к извещению о проведении электрон...,обеспечение исполнения контракта,"{'answer_start': [1228], 'answer_end': [1332],..."
418909970,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1247], 'answer_end': [1317],..."
859637373,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,"{'answer_start': [1246], 'answer_end': [1291],..."


In [ ]:
prepare_bin(train_punctuation_df, option='punctuation')

0it [00:00, ?it/s]

235 items had blank annotation and were skipped


In [ ]:
! python -m spacy init fill-config ./src/model/spacy_cfg.cfg config.cfg

2023-04-21 05:08:55.010029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./punctuation_test/ --paths.train ./punctuation_train_dataset.spacy --paths.dev ./val_dataset.spacy --gpu-id 0 

2023-04-21 05:09:05.202571: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: punctuation_test
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-21 05:09:11,329] [INFO] Set up nlp object from config
[2023-04-21 05:09:11,340] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-21 05:09:11,343] [INFO] Created vocabulary
[2023-04-21 05:09:11,344] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from 

As well as word normalization, punctuation removal drops almost **30% F1-score** on validation set. Also training stability was affected.

In [ ]:
nlp_ner_punctuation = spacy.load('./punctuation_test/model-best')

In [ ]:
test_df.loc[310]

id                                               377776909
text     У Т В Е Р Ж Д А Ю Директор по производству - г...
label                 обеспечение гарантийных обязательств
Name: 310, dtype: object

In [ ]:
doc = nlp_ner_punctuation(test_df.text.loc[310])

colors = {"обеспечение исполнения контракта": "#FF2D00", "обеспечение гарантийных обязательств": "#36FF00"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


This model doesn't perform fine on this document too.

## decap tests


In [ ]:
decap_data_preprocessor = DataPreprocessor(lower_case=True)

In [ ]:
train_decap_df = decap_data_preprocessor(train_df, set_index_to_id=True)

  0%|          | 0/1439 [00:00<?, ?it/s]

In [ ]:
for idx, row in train_decap_df.iterrows():
    train_decap_df.loc[idx].extracted_part['label'] = row.label

In [ ]:
train_decap_df

,text,label,extracted_part
id,,,
351152517,приложение к извещению 1. жетной классификации...,обеспечение исполнения контракта,"{'answer_start': [1026], 'answer_end': [1099],..."
892721516,договор поставки №___________________________ ...,обеспечение гарантийных обязательств,"{'answer_start': [0], 'answer_end': [0], 'text..."
215713905,9 проект государственного контракта № ________...,обеспечение исполнения контракта,"{'answer_start': [1293], 'answer_end': [1407],..."
567219201,извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1277], 'answer_end': [1347],..."
452496091,извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1288], 'answer_end': [1357],..."
...,...,...,...
334674371,приложение №2 к извещению о проведении электро...,обеспечение исполнения контракта,"{'answer_start': [1261], 'answer_end': [1367],..."
418909970,извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,"{'answer_start': [1272], 'answer_end': [1342],..."
859637373,извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,"{'answer_start': [1273], 'answer_end': [1319],..."


In [ ]:
prepare_bin(train_decap_df, option='decap')

0it [00:00, ?it/s]

235 items had blank annotation and were skipped


In [ ]:
! python -m spacy init fill-config ./src/model/spacy_cfg.cfg config.cfg

2023-04-21 06:06:12.029723: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./decap_test/ --paths.train ./decap_train_dataset.spacy --paths.dev ./val_dataset.spacy --gpu-id 0 

2023-04-21 06:06:22.892212: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: decap_test
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-04-21 06:06:29,913] [INFO] Set up nlp object from config
[2023-04-21 06:06:29,924] [INFO] Pipeline: ['transformer', 'ner']
[2023-04-21 06:06:29,927] [INFO] Created vocabulary
[2023-04-21 06:06:29,928] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the ch

Here, F1-score is almost the same to the case with no preprocessing at all. However, there is a small disadvantage of decapitalising because of the fact that the model used is cased one. 

In [ ]:
nlp_ner_decap = spacy.load('./decap_test/model-best')

In [ ]:
test_df.label.loc[310]

'обеспечение гарантийных обязательств'

In [ ]:
doc = nlp_ner_decap(test_df.text.loc[310])

colors = {"обеспечение исполнения контракта": "#FF2D00", "обеспечение гарантийных обязательств": "#36FF00"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
